In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [28]:
import os
import time
import datetime
import pandas as pd
import serial

In [3]:
# テストでBluetooth接続(SSP)から読み取り
with serial.Serial('COM7', 115200, timeout=1) as ser:
    while True:
        c = ser.readline()  # M5Stack側のSerialBT.printの出力を得る
        print(c)

b'710 -264 -110\n'
b'711 -264 -109\n'
b'709 -264 -109\n'
b'711 -262 -108\n'
b'713 -264 -110\n'
b'711 -263 -113\n'
b'714 -262 -109\n'
b'712 -261 -107\n'
b'711 -261 -110\n'
b'709 -266 -112\n'
b'711 -263 -112\n'
b'711 -260 -111\n'
b'710 -261 -110\n'
b'709 -259 -106\n'
b'713 -262 -109\n'
b'713 -262 -110\n'
b'712 -263 -109\n'
b'711 -263 -111\n'
b'710 -264 -111\n'
b'712 -263 -108\n'
b'713 -264 -108\n'
b'711 -264 -107\n'
b'714 -264 -107\n'
b'712 -261 -108\n'
b'711 -262 -110\n'
b'710 -262 -107\n'
b'711 -263 -109\n'
b'711 -264 -111\n'
b'712 -263 -105\n'
b'711 -263 -111\n'
b'713 -263 -109\n'
b'715 -264 -112\n'
b'712 -264 -112\n'
b'712 -263 -112\n'
b'712 -263 -110\n'
b'712 -265 -113\n'
b'711 -266 -114\n'
b'713 -265 -112\n'
b'710 -263 -108\n'
b'711 -263 -108\n'
b'713 -265 -110\n'
b'712 -265 -107\n'
b'713 -263 -109\n'
b'711 -264 -112\n'
b'711 -267 -113\n'
b'711 -267 -110\n'
b'713 -263 -107\n'
b'712 -265 -109\n'
b'712 -264 -108\n'
b'712 -265 -110\n'
b'715 -264 -110\n'
b'711 -264 -109\n'
b'715 -265 -

KeyboardInterrupt: 

In [25]:
class IMUBTRecorder:
    def __init__(self, port, baudrate):
        self.port = port
        self.baudrate = baudrate
        self.columns = ['acceleration_x', 'acceleration_y', 'acceleration_z']
        self.dataframe = pd.DataFrame(columns=self.columns)
        self.start_at = None
        
    def start(self):
        with serial.Serial(self.port, timeout=1, baudrate=self.baudrate) as ser:
            while True:
                data = ser.readline()                         # M5Stack側のSerialBT.printの出力を得る
                data = data.decode(encoding='utf-8')          # バイト列を文字列に変換
                str_data = data.replace('\n', '').split(' ')  # 改行コードを除去してリスト[ax, ay. az]に
                
                # なければロギング終了
                if not str_data:
                    continue
                
                # 計測時間を測る
                now = time.time()
                if not self.start_at:
                    self.start_at = now
                elapsed = now - self.start_at
                
                parsed_data = {k: float(v) for k, v in zip(self.columns, str_data)}
                parsed_data['time'] = elapsed
                self.on_data_received(parsed_data)
                
                # 6分でログを保存して終了(状態遷移のため，20秒ずつ開始と終了を長めにしている)
                if elapsed > 400.0:
                    self.save()
                    break
        
    def on_data_received(self, data):
#         print(data)
        self.make_log(data)
    
    def make_log(self, data):
        s = pd.Series(data)
        self.dataframe = self.dataframe.append(s, ignore_index=True)
        
    def save(self):
        save_path = './dataset/'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        self.dataframe.to_csv(save_path + 'log-{0:%Y-%m-%d-%H-%M-%S}.csv'.format(datetime.datetime.now()))

In [26]:
recorder = IMUBTRecorder('COM7', 115200)
recorder.start()